# Package

In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# 整理營業時間

In [7]:
trip = pd.read_csv("merge_t_g.csv")
print(len(trip["tripadvisor.business_hours"]))
trip.head(3)

1484


,Column1,ID,tri_name,g_name,g_score,tri_score,g_comment,tri_comment,category,address1,tri.address,tripadvisor.link,tripadvisor.business_hours
0,1,538,巴獅子德國餐廳,Der Löwe Bavarian Restaurant 巴獅子德國餐廳,4.4,4.5,788,47,德國料理歐式料理開胃小館,巴獅子德國餐廳00106 台灣大安台北敦化南路二段63巷19號,00106 台灣大安台北敦化南路二段63巷19號,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN
1,2,205,Lacuz Thai Fusion Cuisine Gongguan,Lacuz 泰式雲端廚房,3.5,3.5,1447,367,異國風味亞洲料理泰式料理,Lacuz Thai Fusion Cuisine Gongguan100 台灣大安台北大安...,100 台灣大安台北大安區羅斯福路四段85號2樓,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN
2,3,898,Tibet Kitchen,西藏廚房,4.3,4.5,889,14,印度料理多國料理,Tibet Kitchen100 台灣大安台北和平東路2段217號,100 台灣大安台北和平東路2段217號,https://www.tripadvisor.com.tw/Restaurant_Revi...,日11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 一11:00 ...


In [8]:
trip["Monday"] = [np.nan]*1484
trip["Tuesday"] = [np.nan]*1484
trip["Wednesday"] = [np.nan]*1484
trip["Thursday"] = [np.nan]*1484
trip["Friday"] = [np.nan]*1484
trip["Saturday"] = [np.nan]*1484
trip["Sunday"] = [np.nan]*1484

trip.head(3)

,Column1,ID,tri_name,g_name,g_score,tri_score,g_comment,tri_comment,category,address1,tri.address,tripadvisor.link,tripadvisor.business_hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1,538,巴獅子德國餐廳,Der Löwe Bavarian Restaurant 巴獅子德國餐廳,4.4,4.5,788,47,德國料理歐式料理開胃小館,巴獅子德國餐廳00106 台灣大安台北敦化南路二段63巷19號,00106 台灣大安台北敦化南路二段63巷19號,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,205,Lacuz Thai Fusion Cuisine Gongguan,Lacuz 泰式雲端廚房,3.5,3.5,1447,367,異國風味亞洲料理泰式料理,Lacuz Thai Fusion Cuisine Gongguan100 台灣大安台北大安...,100 台灣大安台北大安區羅斯福路四段85號2樓,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,898,Tibet Kitchen,西藏廚房,4.3,4.5,889,14,印度料理多國料理,Tibet Kitchen100 台灣大安台北和平東路2段217號,100 台灣大安台北和平東路2段217號,https://www.tripadvisor.com.tw/Restaurant_Revi...,日11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 一11:00 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
hour = trip["tripadvisor.business_hours"][0]
print(hour)

日11:30 AM - 9:30 PM 一11:30 AM - 2:30 PM 5:30 PM - 9:30 PM 二11:30 AM - 2:30 PM 5:30 PM - 9:30 PM 三11:30 AM - 2:30 PM 5:30 PM - 9:30 PM 四05:30 AM - 9:30 PM 五11:30 AM - 2:30 PM 5:30 PM - 10:00 PM 六11:30 AM - 2:30 PM 5:30 PM - 10:00 PM


In [28]:
#hour = re.sub(" - ", "-", trip["tripadvisor.business_hours"][0])
hour_list = list(filter(None, re.split(r'(日|一|二|三|四|五|六)', hour)))
hour_list = ["".join(i) for i in zip(hour_list[0::2],hour_list[1::2])]
print(hour_list)

['日11:30 AM - 9:30 PM ', '一11:30 AM - 2:30 PM 5:30 PM - 9:30 PM ', '二11:30 AM - 2:30 PM 5:30 PM - 9:30 PM ', '三11:30 AM - 2:30 PM 5:30 PM - 9:30 PM ', '四05:30 AM - 9:30 PM ', '五11:30 AM - 2:30 PM 5:30 PM - 10:00 PM ', '六11:30 AM - 2:30 PM 5:30 PM - 10:00 PM']


In [44]:
for hour in hour_list:
    if "一" in hour:
        trip["Monday"][0] = re.sub("一", "", hour)
    elif "二" in hour:
        trip["Tuesday"][0] = re.sub("二", "", hour)
    elif "三" in hour:
        trip["Wednesday"][0] = re.sub("三", "", hour)
    elif "四" in hour:
        trip["Thursday"][0] = re.sub("四", "", hour)
    elif "五" in hour:
        trip["Friday"][0] = re.sub("五", "", hour)
    elif "六" in hour:
        trip["Saturday"][0] = re.sub("六", "", hour)
    elif "日" in hour:
        trip["Sunday"][0] = re.sub("日", "", hour)
trip.head(3)

/Users/liuchenli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/liuchenli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liuchenli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

,ID,tri_name,g_name,g_score,tri_score,g_comment,tri_comment,category,address1,tri.address,tripadvisor.link,tripadvisor.business_hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,食令Shabu,食令shabu ∣ 信義區頂級鍋物 台北101購物中心,4.7,5.0,"1,815",237,海鮮,食令Shabu110 台灣台北台北市信義區市府路45號4F 台北101,110 台灣台北台北市信義區市府路45號4F 台北101,https://www.tripadvisor.com.tw/Restaurant_Revi...,日11:30 AM - 9:30 PM 一11:30 AM - 2:30 PM 5:30 P...,11:30 AM - 2:30 PM 5:30 PM - 9:30 PM,11:30 AM - 2:30 PM 5:30 PM - 9:30 PM,11:30 AM - 2:30 PM 5:30 PM - 9:30 PM,05:30 AM - 9:30 PM,11:30 AM - 2:30 PM 5:30 PM - 10:00 PM,11:30 AM - 2:30 PM 5:30 PM - 10:00 PM,11:30 AM - 9:30 PM
1,1,肉老大頂級肉品涮涮鍋- 敦南店,肉老大頂級肉品涮涮鍋 台北敦南店,4.8,5.0,"3,605",310,中式料理亞洲料理熟食,肉老大頂級肉品涮涮鍋- 敦南店106 台灣大安台北敦化南路二段331巷16號,106 台灣大安台北敦化南路二段331巷16號,https://www.tripadvisor.com.tw/Restaurant_Revi...,日11:30 AM - 11:45 PM 一11:30 AM - 4:00 PM 4:30 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,合.Shabu合鍋物,合.shabu ∣ 信義區頂級鍋物,4.6,4.5,"3,307",1911,海鮮亞洲料理熟食,"合.Shabu合鍋物110 台灣信義區台北No.28, Songren Rd., Xinyi...","110 台灣信義區台北No.28, Songren Rd., Xinyi Dist Bell...",https://www.tripadvisor.com.tw/Restaurant_Revi...,日12:00 AM - 2:30 PM 5:30 PM - 10:00 PM 一12:00 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
print(trip["tripadvisor.business_hours"][8])
print(type(trip["tripadvisor.business_hours"][8]))

nan
<class 'float'>


In [9]:
for j in tqdm(range(len(trip["tripadvisor.business_hours"]))):
    hour = trip["tripadvisor.business_hours"][j]
    if type(hour) != float:
        hour_list = list(filter(None, re.split(r'(日|一|二|三|四|五|六)', hour)))
        hour_list = ["".join(i) for i in zip(hour_list[0::2],hour_list[1::2])]
        for hour in hour_list:
            if "一" in hour:
                trip["Monday"][j] = re.sub("一", "", hour)
            elif "二" in hour:
                trip["Tuesday"][j] = re.sub("二", "", hour)
            elif "三" in hour:
                trip["Wednesday"][j] = re.sub("三", "", hour)
            elif "四" in hour:
                trip["Thursday"][j] = re.sub("四", "", hour)
            elif "五" in hour:
                trip["Friday"][j] = re.sub("五", "", hour)
            elif "六" in hour:
                trip["Saturday"][j] = re.sub("六", "", hour)
            elif "日" in hour:
                trip["Sunday"][j] = re.sub("日", "", hour)
trip.head(3)

/Users/liuchenli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/liuchenli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/liuchenli/.local/l

,Column1,ID,tri_name,g_name,g_score,tri_score,g_comment,tri_comment,category,address1,tri.address,tripadvisor.link,tripadvisor.business_hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1,538,巴獅子德國餐廳,Der Löwe Bavarian Restaurant 巴獅子德國餐廳,4.4,4.5,788,47,德國料理歐式料理開胃小館,巴獅子德國餐廳00106 台灣大安台北敦化南路二段63巷19號,00106 台灣大安台北敦化南路二段63巷19號,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,205,Lacuz Thai Fusion Cuisine Gongguan,Lacuz 泰式雲端廚房,3.5,3.5,1447,367,異國風味亞洲料理泰式料理,Lacuz Thai Fusion Cuisine Gongguan100 台灣大安台北大安...,100 台灣大安台北大安區羅斯福路四段85號2樓,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,898,Tibet Kitchen,西藏廚房,4.3,4.5,889,14,印度料理多國料理,Tibet Kitchen100 台灣大安台北和平東路2段217號,100 台灣大安台北和平東路2段217號,https://www.tripadvisor.com.tw/Restaurant_Revi...,日11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 一11:00 ...,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM


In [10]:
trip.to_csv(("merge_t_g_open.csv"), index=False, encoding="utf-8")

# 爬取一家餐廳：價位與菜系

In [2]:
URL = "https://www.tripadvisor.com.tw/Restaurant_Review-g13808515-d2202371-Reviews-Yen_Chinese_Restaurant-Xinyi_District_Taipei.html"

response = requests.get(URL)
response.encoding = "UTF-8"
response.status_code

soup = BeautifulSoup(response.text,"html.parser")

In [3]:
# 補爬價位與菜系
infos = soup.find_all("div", class_ = "tbUiL b")
details = soup.find_all("div", class_ = "SrqKb")
info_list = []
detail_list = []
for info in infos:
    info_list.append(info.text)
for detail in details:
    detail_list.append(detail.text)

print(info_list)
print(detail_list)

['價位', '菜系', '特別飲食限制']
['NT$1,078 - NT$5,330', '中式料理, 亞洲料理', '提供素食選擇, 提供純素選擇, 提供無麩質選擇']


In [4]:
try:
    price_loc = info_list.index("價位")
    price = re.sub(",| |[NT$]", "", detail_list[price_loc])
    price = re.split(r'[-]', price)
    if len(price)==2:
        ave_price = int((int(price[1])-int(price[0]))/2)
    else:
        ave_price = int(price)
except:
    ave_price = np.nan

try:
    category_loc = info_list.index("菜系")
    category = re.sub(" ", "", detail_list[category_loc])
except:
    category = np.nan

print(ave_price)
print(category)

2126
中式料理,亞洲料理


# 爬取所有餐廳：價位與菜系

In [5]:
trip = pd.read_csv("merge_t_g_open.csv")
t_link = trip["tripadvisor.link"]
print(len(t_link))

1484


In [71]:
trip["ave_price"] = [np.nan]*1484
trip["category"] = [np.nan]*1484

In [ ]:
# 補爬價位與菜系
for i in tqdm(range(len(t_link))):

    URL = t_link[i]
    response = requests.get(URL)
    response.encoding = "UTF-8"
    soup = BeautifulSoup(response.text,"html.parser")
    
    infos = soup.find_all("div", class_ = "tbUiL b")
    details = soup.find_all("div", class_ = "SrqKb")
    info_list = []
    detail_list = []
    for info in infos:
        info_list.append(info.text)
    for detail in details:
        detail_list.append(detail.text)

    try:
        price_loc = info_list.index("價位")
        price = re.sub(",| |[NT$]", "", detail_list[price_loc])
        price = re.split(r'[-]', price)
        if len(price)==2:
            ave_price = int((int(price[1])-int(price[0]))/2)
        else:
            ave_price = int(price)
    except:
        ave_price = np.nan

    try:
        category_loc = info_list.index("菜系")
        category = re.sub(" ", "", detail_list[category_loc])
    except:
        category = np.nan

    trip["ave_price"][i] = ave_price
    trip["category"][i] = category
    
    trip.to_csv(("merge_t_g_price.csv"), index=False, encoding="utf-8")

    if i%15==0:
        time.sleep(60)

# 再次整理營業時間

In [2]:
trip = pd.read_csv("merge_t_g_price.csv")
print(len(trip["tripadvisor.business_hours"]))
trip.head(3)

1484


,Column1,ID,tri_name,g_name,g_score,tri_score,g_comment,tri_comment,category,address1,...,tripadvisor.business_hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,ave_price,category_new
0,1,538,巴獅子德國餐廳,Der Löwe Bavarian Restaurant 巴獅子德國餐廳,4.4,4.5,788,47,德國料理歐式料理開胃小館,巴獅子德國餐廳00106 台灣大安台北敦化南路二段63巷19號,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308.0,"德國料理,歐式料理,開胃小館,中歐料理"
1,2,205,Lacuz Thai Fusion Cuisine Gongguan,Lacuz 泰式雲端廚房,3.5,3.5,1447,367,異國風味亞洲料理泰式料理,Lacuz Thai Fusion Cuisine Gongguan100 台灣大安台北大安...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175.0,"異國風味,亞洲料理,泰式料理"
2,3,898,Tibet Kitchen,西藏廚房,4.3,4.5,889,14,印度料理多國料理,Tibet Kitchen100 台灣大安台北和平東路2段217號,...,日11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 一11:00 ...,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,11:00 AM - 2:30 PM 5:00 PM - 10:30 PM,95.0,"印度料理,多國料理"


In [3]:
trip["Monday"] = [np.nan]*1484
trip["Tuesday"] = [np.nan]*1484
trip["Wednesday"] = [np.nan]*1484
trip["Thursday"] = [np.nan]*1484
trip["Friday"] = [np.nan]*1484
trip["Saturday"] = [np.nan]*1484
trip["Sunday"] = [np.nan]*1484

trip.head(3)

,Column1,ID,tri_name,g_name,g_score,tri_score,g_comment,tri_comment,category,address1,...,tripadvisor.business_hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,ave_price,category_new
0,1,538,巴獅子德國餐廳,Der Löwe Bavarian Restaurant 巴獅子德國餐廳,4.4,4.5,788,47,德國料理歐式料理開胃小館,巴獅子德國餐廳00106 台灣大安台北敦化南路二段63巷19號,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308.0,"德國料理,歐式料理,開胃小館,中歐料理"
1,2,205,Lacuz Thai Fusion Cuisine Gongguan,Lacuz 泰式雲端廚房,3.5,3.5,1447,367,異國風味亞洲料理泰式料理,Lacuz Thai Fusion Cuisine Gongguan100 台灣大安台北大安...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175.0,"異國風味,亞洲料理,泰式料理"
2,3,898,Tibet Kitchen,西藏廚房,4.3,4.5,889,14,印度料理多國料理,Tibet Kitchen100 台灣大安台北和平東路2段217號,...,日11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 一11:00 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,"印度料理,多國料理"


In [12]:
hour = trip["tripadvisor.business_hours"][2]
print(hour)

日11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 一11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 二11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 三11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 四11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 五11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 六11:00 AM - 2:30 PM 5:00 PM - 10:30 PM


In [13]:
hour = re.sub(" - ", "-", hour)
hour = re.sub(" A", "A", hour)
hour = re.sub(" P", "P", hour)
hour = re.sub(" ", ", ", hour)
hour_list = list(filter(None, re.split(r'(日|一|二|三|四|五|六)', hour)))
hour_list = ["".join(i) for i in zip(hour_list[0::2],hour_list[1::2])]
hour_list = [i.rstrip(", ") for i in hour_list]
print(hour_list)

['日11:00AM-2:30PM, 5:00PM-10:30PM', '一11:00AM-2:30PM, 5:00PM-10:30PM', '二11:00AM-2:30PM, 5:00PM-10:30PM', '三11:00AM-2:30PM, 5:00PM-10:30PM', '四11:00AM-2:30PM, 5:00PM-10:30PM', '五11:00AM-2:30PM, 5:00PM-10:30PM', '六11:00AM-2:30PM, 5:00PM-10:30PM']


In [9]:
for hour in hour_list:
    if "一" in hour:
        trip["Monday"][2] = re.sub("一", "", hour)
    elif "二" in hour:
        trip["Tuesday"][2] = re.sub("二", "", hour)
    elif "三" in hour:
        trip["Wednesday"][2] = re.sub("三", "", hour)
    elif "四" in hour:
        trip["Thursday"][2] = re.sub("四", "", hour)
    elif "五" in hour:
        trip["Friday"][2] = re.sub("五", "", hour)
    elif "六" in hour:
        trip["Saturday"][2] = re.sub("六", "", hour)
    elif "日" in hour:
        trip["Sunday"][2] = re.sub("日", "", hour)
trip.head(3)

/Users/liuchenli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/liuchenli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

,Column1,ID,tri_name,g_name,g_score,tri_score,g_comment,tri_comment,category,address1,...,tripadvisor.business_hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,ave_price,category_new
0,1,538,巴獅子德國餐廳,Der Löwe Bavarian Restaurant 巴獅子德國餐廳,4.4,4.5,788,47,德國料理歐式料理開胃小館,巴獅子德國餐廳00106 台灣大安台北敦化南路二段63巷19號,...,NaN,"11:00AM-2:30PM, 5:00PM-10:30PM,","11:00AM-2:30PM, 5:00PM-10:30PM,","11:00AM-2:30PM, 5:00PM-10:30PM,","11:00AM-2:30PM, 5:00PM-10:30PM,","11:00AM-2:30PM, 5:00PM-10:30PM,","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM,",308.0,"德國料理,歐式料理,開胃小館,中歐料理"
1,2,205,Lacuz Thai Fusion Cuisine Gongguan,Lacuz 泰式雲端廚房,3.5,3.5,1447,367,異國風味亞洲料理泰式料理,Lacuz Thai Fusion Cuisine Gongguan100 台灣大安台北大安...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175.0,"異國風味,亞洲料理,泰式料理"
2,3,898,Tibet Kitchen,西藏廚房,4.3,4.5,889,14,印度料理多國料理,Tibet Kitchen100 台灣大安台北和平東路2段217號,...,日11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 一11:00 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,"印度料理,多國料理"


In [14]:
for j in tqdm(range(len(trip["tripadvisor.business_hours"]))):
    hour = trip["tripadvisor.business_hours"][j]
    if type(hour) != float:
        hour = re.sub(" - ", "-", hour)
        hour = re.sub(" A", "A", hour)
        hour = re.sub(" P", "P", hour)
        hour = re.sub(" ", ", ", hour)
        hour_list = list(filter(None, re.split(r'(日|一|二|三|四|五|六)', hour)))
        hour_list = ["".join(i) for i in zip(hour_list[0::2],hour_list[1::2])]
        hour_list = [i.rstrip(", ") for i in hour_list]
        for hour in hour_list:
            if "一" in hour:
                trip["Monday"][j] = re.sub("一", "", hour)
            elif "二" in hour:
                trip["Tuesday"][j] = re.sub("二", "", hour)
            elif "三" in hour:
                trip["Wednesday"][j] = re.sub("三", "", hour)
            elif "四" in hour:
                trip["Thursday"][j] = re.sub("四", "", hour)
            elif "五" in hour:
                trip["Friday"][j] = re.sub("五", "", hour)
            elif "六" in hour:
                trip["Saturday"][j] = re.sub("六", "", hour)
            elif "日" in hour:
                trip["Sunday"][j] = re.sub("日", "", hour)
trip.head(3)

/Users/liuchenli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/liuchenli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/li

,Column1,ID,tri_name,g_name,g_score,tri_score,g_comment,tri_comment,category,address1,...,tripadvisor.business_hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,ave_price,category_new
0,1,538,巴獅子德國餐廳,Der Löwe Bavarian Restaurant 巴獅子德國餐廳,4.4,4.5,788,47,德國料理歐式料理開胃小館,巴獅子德國餐廳00106 台灣大安台北敦化南路二段63巷19號,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308.0,"德國料理,歐式料理,開胃小館,中歐料理"
1,2,205,Lacuz Thai Fusion Cuisine Gongguan,Lacuz 泰式雲端廚房,3.5,3.5,1447,367,異國風味亞洲料理泰式料理,Lacuz Thai Fusion Cuisine Gongguan100 台灣大安台北大安...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175.0,"異國風味,亞洲料理,泰式料理"
2,3,898,Tibet Kitchen,西藏廚房,4.3,4.5,889,14,印度料理多國料理,Tibet Kitchen100 台灣大安台北和平東路2段217號,...,日11:00 AM - 2:30 PM 5:00 PM - 10:30 PM 一11:00 ...,"11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM",95.0,"印度料理,多國料理"


In [15]:
trip.to_csv(("merge_t_g_open_price.csv"), index=False, encoding="utf-8")

# 爬取一家餐廳：價位與圖片

In [3]:
all = pd.read_csv("merge_all_new.csv")
all.head(3)

,Unnamed: 0,contain,name_g,score_g,num_g,score_i,num_i,score_t,num_t,address,...,Wednesday,Thursday,Friday,Saturday,Sunday,image_source,link_i,link_t,name_i,name_t
0,1,two,西藏廚房,4.3,889,NaN,NaN,4.5,14.0,100 台灣大安台北和平東路2段217號,...,"11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM",NaN,NaN,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,Tibet Kitchen
1,2,two,HOW&THE JUICE,4.9,137,NaN,NaN,4.0,1.0,"106 台灣大安台北No 97, Section 4, Zhongxiao East Roa...",...,11:30AM-8:30PM,11:30AM-8:30PM,11:30AM-8:30PM,11:30AM-8:30PM,11:30AM-8:30PM,NaN,NaN,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,HOW & the JUICE
2,3,two,CAMPUS CAFE 忠孝店,3.9,3530,NaN,NaN,4.0,147.0,"106 台灣大安台北No. 203, Section 3, Zhongxiao East R...",...,3:00PM-10:00PM,3:00PM-10:00PM,3:00PM-10:00PM,3:00PM-10:00PM,休息,NaN,NaN,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,Campus Cafe


In [3]:
i = 0
all.iloc[i,]

Unnamed: 0                                                      1
contain                                                       two
name_g                                                       西藏廚房
score_g                                                       4.3
num_g                                                         889
score_i                                                       NaN
num_i                                                         NaN
score_t                                                       4.5
num_t                                                        14.0
address                                      100 台灣大安台北和平東路2段217號
區                                                             大安區
路                                                             NaN
段                                                             NaN
cate_i                                                        NaN
cate_t                                                  印度料理,多國料理
ave_price 

In [4]:
# test string type nan
print(all["link_t"][1488])
type(all["link_t"][1488])

nan


float

In [26]:
# test numeric type nan
print(all["ave_price"][i])
np.isnan(all["ave_price"][i])

nan


True

In [28]:
print(all["ave_price"][1488])
np.isnan(all["ave_price"][1488])

150.0


False

In [33]:
# test string type nan
print(all["image_file"][i])
type(all["image_file"][i])

nan


float

In [36]:
np.isnan(all["ave_price"][i]) or type(all["image_file"][i]) == float

True

In [5]:
i = 1
driver = webdriver.Chrome(ChromeDriverManager().install())
#driver = webdriver.Chrome("chromedriver")

if type(all["link_t"][i]) != float:

    if np.isnan(all["ave_price"][i]) or type(all["image_file"][i]) == float:
        URL = all["link_t"][i]
        driver.get(URL)
        driver.implicitly_wait(10)
        soup = BeautifulSoup(driver.page_source,"html.parser")
    
    # 價格
    if np.isnan(all["ave_price"][i]):
    
        infos = soup.find_all("div", class_ = "tbUiL b")
        details = soup.find_all("div", class_ = "SrqKb")
        info_list = []
        detail_list = []
        for info in infos:
            info_list.append(info.text)
        for detail in details:
            detail_list.append(detail.text)

        try:
            price_loc = info_list.index("價位")
            price = re.sub(",| |[NT$]", "", detail_list[price_loc])
            price = re.split(r'[-]', price)
            if len(price)==2:
                min_price = int(price[0])
                max_price = int(price[1])
                ave_price = int((max_price+min_price)/2)
            else:
                ave_price = int(price)
        except:
            min_price = np.nan
            max_price = np.nan
            ave_price = np.nan

    # 下載封面圖檔
    if type(all["image_file"][i]) == float:

        image_id = "tripadvisor_"+str(i)+".png"
        cover = soup.find("div", class_ = "prw_rup prw_common_basic_image photo_widget large landscape")
        image_link = cover.img["data-lazyurl"]
        
        img = requests.get(image_link).content
        with open("image/"+image_id, 'wb') as handler:
            handler.write(img)

print(min_price, max_price, ave_price, image_id, image_link)

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:01<00:00, 4.49MB/s]
C:\Users\AJ100-MAR-2022\AppData\Local\Temp\ipykernel_30704\2936184619.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


nan nan nan tripadvisor_1.png https://media-cdn.tripadvisor.com/media/photo-s/0f/72/bb/e0/sign.jpg


In [6]:
driver.quit()

# 爬取所有餐廳：價位與圖片

In [2]:
all = pd.read_csv("merge_all_price9.csv")
all.head(3)

,Unnamed: 0,contain,name_g,score_g,num_g,score_i,num_i,score_t,num_t,address,...,Friday,Saturday,Sunday,image_source,link_i,link_t,name_i,name_t,max_price,min_price
0,1,two,西藏廚房,4.3,889,NaN,NaN,4.5,14.0,100 台灣大安台北和平東路2段217號,...,"11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM","11:00AM-2:30PM, 5:00PM-10:30PM",https://media-cdn.tripadvisor.com/media/photo-...,NaN,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,Tibet Kitchen,290.0,100.0
1,2,two,HOW&THE JUICE,4.9,137,NaN,NaN,4.0,1.0,"106 台灣大安台北No 97, Section 4, Zhongxiao East Roa...",...,11:30AM-8:30PM,11:30AM-8:30PM,11:30AM-8:30PM,https://media-cdn.tripadvisor.com/media/photo-...,NaN,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,HOW & the JUICE,NaN,NaN
2,3,two,CAMPUS CAFE 忠孝店,3.9,3530,NaN,NaN,4.0,147.0,"106 台灣大安台北No. 203, Section 3, Zhongxiao East R...",...,3:00PM-10:00PM,3:00PM-10:00PM,休息,https://media-cdn.tripadvisor.com/media/photo-...,NaN,https://www.tripadvisor.com.tw/Restaurant_Revi...,NaN,Campus Cafe,NaN,NaN


In [3]:
len(all["name_g"])

3177

In [4]:
#all["max_price"] = [np.nan]*3177
#all["min_price"] = [np.nan]*3177

In [4]:
#driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome("chromedriver")
session = requests.Session()

j = 0

for i in tqdm(range(796,len(all["name_g"]))):

    if type(all["link_t"][i]) != float and (np.isnan(all["ave_price"][i]) or type(all["image_file"][i]) == float):

        URL = all["link_t"][i]
        #driver = webdriver.Chrome("chromedriver")
        driver.get(URL)
        driver.implicitly_wait(10)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        j += 1
        
        # 價格
        if np.isnan(all["ave_price"][i]):
        
            infos = soup.find_all("div", class_ = "tbUiL b")
            details = soup.find_all("div", class_ = "SrqKb")
            info_list = []
            detail_list = []
            for info in infos:
                info_list.append(info.text)
            for detail in details:
                detail_list.append(detail.text)

            try:
                price_loc = info_list.index("價位")
                price = re.sub(",| |[NT$]", "", detail_list[price_loc])
                price = re.split(r'[-]', price)
                if len(price)==2:
                    min_price = int(price[0])
                    max_price = int(price[1])
                    ave_price = int((max_price+min_price)/2)
                else:
                    ave_price = int(price)
            except:
                min_price = np.nan
                max_price = np.nan
                ave_price = np.nan

        # 下載封面圖檔
        if type(all["image_file"][i]) == float:
            
            try:
                image_id = "tripadvisor_"+str(i)+".png"
                cover = soup.find("div", class_ = "prw_rup prw_common_basic_image photo_widget large landscape")
                image_link = cover.img["data-lazyurl"]

                img_response = session.get(image_link)
                img = img_response.content
                with open("image/"+image_id, 'wb') as handler:
                    handler.write(img)

            except:
                image_id = np.nan
                image_link = np.nan

        all["min_price"][i] = min_price
        all["max_price"][i] = max_price
        all["ave_price"][i] = ave_price
        all["image_file"][i] = image_id
        all["image_source"][i] = image_link
        
        all.to_csv(("merge_all_price10.csv"), index=False, encoding="utf-8")

        if j%15==0:
            time.sleep(60)
        
        #driver.quit()
    
    else:
        continue

driver.quit()

  0%|          | 0/2381 [00:00<?, ?it/s]

C:\Users\AJ100-MAR-2022\AppData\Local\Temp\ipykernel_31396\2206400145.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all["min_price"][i] = min_price
C:\Users\AJ100-MAR-2022\AppData\Local\Temp\ipykernel_31396\2206400145.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all["max_price"][i] = max_price
C:\Users\AJ100-MAR-2022\AppData\Local\Temp\ipykernel_31396\2206400145.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all["a